In [26]:
import numpy as np
import pandas as pd

data = pd.read_csv(r'dados/movie_metadata.csv')

data.drop_duplicates()
data.values

array([['Color', 'James Cameron', 723.0, ..., 7.9, 1.78, 33000],
       ['Color', 'Gore Verbinski', 302.0, ..., 7.1, 2.35, 0],
       ['Color', 'Sam Mendes', 602.0, ..., 6.8, 2.35, 85000],
       ...,
       ['Color', 'Benjamin Roberds', 13.0, ..., 6.3, nan, 16],
       ['Color', 'Daniel Hsia', 14.0, ..., 6.3, 2.35, 660],
       ['Color', 'Jon Gunn', 43.0, ..., 6.6, 1.85, 456]], dtype=object)

# Consultas

In [2]:
# Consulta 1: Filme com a maior arrecadação
def consulta1(data):
    aux = data[['movie_title', 'gross'][:]].dropna()
    return pd.DataFrame(aux.loc[aux.gross.idxmax()])

# Consulta 2: Filme com maior arrecadação para cada idioma
def consulta2(data):
    aux = data[['language', 'movie_title', 'gross'][:]].dropna()
    return aux.reindex(labels=aux.groupby('language').gross.idxmax(), axis="index")

# Consulta 3: Top 3 países com a maior quantidade de filmes
def consulta3(data):
    result = []
    aux = data[['country', 'movie_title'][:]].dropna()
    country_list = aux.groupby('country').movie_title.nunique().to_frame()
    for i in range(3):
        result.append(country_list.loc[country_list.movie_title.idxmax()])
        country_list = country_list.drop(country_list.movie_title.idxmax())
    return pd.DataFrame(result)

# consulta1(data)
# consulta2(data)
# consulta3(data)

# Funções Score

In [3]:
def score1(data, out):
    values = []
    for i in data:
        if(i.movie_title != 'None' and not np.isnan(i.gross)):
            if(i.gross == out):
                values.append(i.gross)
            else:
                values.append(0)
    return values

def score2(data, out):
    values = []
    for i in data:
        if(i in out and not np.isnan(i.gross)):
            values.append(i.gross)
        else:
            values.append(0)
    return values

def score3(data, out):
    data_countries = data.country.dropna().array
    countries, values = []
    for i in range(0, len(data_countries)):
        if(data_countries[i] not in countries):
            countries[data_countries[i]] = 0
    for i in data:
        if(i.country != 'None'):
            countries[i.country] += 1
            if(i.country in saida):
                values.append(i.country)
            else:
                values.append(0)
    return values

# Funções de sensibilidade

In [32]:
def sense_q1(data):
    mat = []
    result = data
    for i in result:
        print(i)
        mat.append(score1(data, i))
    sense = 0
    for i in range(0, len(mat)):
        for j in range(0, len(data) - 1):
            for k in range(j + 1, len(data)):
                if(abs(mat[i][j] - mat[i][k] >= sense)):
                    sense = abs(mat[i][j] - mat[i][k])
    return sense

sense_q1(data)

color
color


AttributeError: 'str' object has no attribute 'movie_title'